<a href="https://colab.research.google.com/github/Viktory4121/MORD/blob/main/%D0%94%D0%9E%D0%9C.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Динамические окрестностные модели (ДОМ)
####Вариант 5

In [1]:
import numpy as np
import pandas as pd
import random
import copy
import math
from prettytable import PrettyTable
import itertools as it

In [2]:
#A={a1, a2, a3}

#какие дуги входят в позицию:
#Ox[a1]={a1}
#Ox[a2]={}
#Ox[a3]={a1, a2}

#в какие позиции входят v
#Ov[a1]={a1}
#Ov[a2]={a2}
#Ov[a3]={a3}

#количества состояний/входов/выходов
elem = 100
count = [3, 3, 3]
Sx = np.array([[1, 0, 1], [0, 0, 1], [0, 0, 0]])
Sv = np.array([[1, 0, 0], [0, 1, 0], [0, 0, 1]])

print("Матрица смежности:")
print("Sx = ")
print(Sx)
print("Sv = ")
print(Sv)

Матрица смежности:
Sx = 
[[1 0 1]
 [0 0 1]
 [0 0 0]]
Sv = 
[[1 0 0]
 [0 1 0]
 [0 0 1]]


In [3]:
def x1(x, v):
  return math.sin(math.log(abs(v[0])) / x[0])

def x2(x, v):
  return math.cos(v[1]**3)

def x3(x, v):
  return math.sin(math.tan(x[0] / x[1]) - 3**(v[2]))

def v1(x, v):
  return (v[0] + math.tan(x[0])**2)

def v2(x, v):
  return (0.2 * v[1])

def v3(x, v):
  return (math.exp(x[0]) + .45 * math.log2(abs(x[1] * v[2])))

def y1(x, v):
  return (x[0] / v[0]) * math.sqrt(abs(x[0]))

def y2(x, v):
  return math.sqrt(abs(math.sin(v[1])))

def y3(x, v):
  return v[2] * x[0] / (x[1] + 0.1)

functions = [x1, x2, x3, v1, v2, v3, y1, y2, y3]

In [4]:
#Метод псевдообращения Бена-Израиля
def BenIsrael(A, eps = 0.000001): #А - np.array, eps - точность вычислительная
  I = np.diag(np.diag(np.ones([len(A),len(A)])))
  buf = 1
  i = 1

  if(np.linalg.norm(A, ord='fro') != 0): #если не нулевая норма
    gamma = 1.6 / (np.linalg.norm(A, ord='fro')**2)
    A1 = gamma * (A.T)
    i += 1
    A2 = A1.dot(2*I - A.dot(A1))
    E = np.linalg.norm(A2 - A1, ord=np.inf) / np.linalg.norm(A1, ord=np.inf)
  else: #если нулевая норма
    A1 = np.zeros((len(A[0]), len(A)))
    buf = 0

  while(E >= eps and buf):
    A1 = A2.copy()
    A2 = A1.dot(2*I - A.dot(A1))
    E = np.linalg.norm(A2 - A1, ord=np.inf) / np.linalg.norm(A1, ord=np.inf)
    i += 1

  return A1

#решение СЛАУ вида AX=B
def solvingSLAE(A, B):
  A = np.array(A.copy())

  #проверка на бесконечное количество решений
  A_psev = BenIsrael(A)

  X = A_psev.dot(B)
  return X

#=================================================================
#Генерация случайного вектора из 3-х элементов в диапазоне от -3 до 3
def generationRandomVector(elem = 3, start = -3, end = 3):
  vector = []
  for i in range(elem):
    vector.append(random.uniform(start, end))
  return vector

#=================================================================
#создание столбца таблицы
def createVariants(function, x, v, typeFunc = 1, elem = 100):
  vector = []

  if(typeFunc == 0):
    for i in range(elem):
      vector.append(function(generationRandomVector(), generationRandomVector()))
  else:
    for i in range(elem):
      vector.append(function(x[i], v[i]))

  return vector

#=================================================================
#Создание ДОМ в определённый момент времени
def createTableDNM(functions, count, step, oldTable = []): #count(x, v, y)
  table = []
  vector = []
  columns = createColumns(step, count)

  if(step == 0):
    for i in range(sum(count[0:2])):
      table.append(createVariants(functions[i], [], [], 0)) #xi, vi

  else:
    for i in range(count[0]): #xi
      table.append(createVariants(functions[i], (np.array(oldTable))[:, 0:count[0]], (np.array(oldTable))[:, count[0]:sum(count[0:2])]))
    for i in range(count[1]): #vi
      table.append(createVariants(functions[count[0] + i], [], [], 0))

  for i in range(count[2]): #yi
    table.append(createVariants(functions[sum(count[0:2]) + i], (np.array(table).T)[:, 0:count[0]], (np.array(table).T)[:, count[0]:sum(count[0:2])]))

  table = pd.DataFrame(np.array(table).T, columns = columns)
  return table

#заполнение
def createColumns(step, count):
  columns = []
  for i in range(count[0]):
    columns.append("x" + str(i + 1) + "(" + str(step) + ")")
  for i in range(count[1]):
    columns.append("v" + str(i + 1) + "(" + str(step) + ")")
  for i in range(count[2]):
    columns.append("y" + str(i + 1) + "(" + str(step) + ")")
  return columns

In [5]:
#functions = [x1, x2, x3, v1, v2, v3, y1, y2, y3]
model_0 = createTableDNM(functions, count, 0)
model_1 = createTableDNM(functions, count, 1, np.array(model_0))
model_2 = createTableDNM(functions, count, 2, np.array(model_1))
model_3 = createTableDNM(functions, count, 3, np.array(model_2))
model_4 = createTableDNM(functions, count, 4, np.array(model_3))
model_5 = createTableDNM(functions, count, 5, np.array(model_4))
model_6 = createTableDNM(functions, count, 6, np.array(model_5))
model_7 = createTableDNM(functions, count, 7, np.array(model_6))
model_8 = createTableDNM(functions, count, 8, np.array(model_7))
model_9 = createTableDNM(functions, count, 9, np.array(model_8))
model_10 = createTableDNM(functions, count, 10, np.array(model_9))

In [6]:
model_9

,x1(9),x2(9),x3(9),v1(9),v2(9),v3(9),y1(9),y2(9),y3(9)
0,0.951079,0.995348,0.989685,6.477269,-0.453093,0.829883,0.143197,0.661625,0.720578
1,0.885121,0.991708,0.071718,31.089224,-0.050094,8.085761,0.026785,0.223769,6.555670
2,-0.056873,0.999978,-0.874990,1.608868,-0.443205,4.950907,-0.008430,0.654856,-0.255981
3,-0.741657,0.995284,-0.356255,1.446732,-0.194096,1.414183,-0.441486,0.439180,-0.957596
4,0.194237,0.998147,-0.983286,3.557893,0.232657,-0.012260,0.024061,0.480170,-0.002169
...,...,...,...,...,...,...,...,...,...
95,0.483175,1.000000,-0.699493,5.194038,0.299201,1.047673,0.064662,0.542915,0.460191
96,-0.936859,0.999975,-0.457414,177.883398,0.210993,1.171087,-0.005098,0.457636,-0.997425
97,0.139984,0.999856,0.678799,-1.675065,-0.490576,7.887361,-0.031267,0.686392,1.003866
98,0.584783,0.979541,-0.665780,-0.639987,-0.326581,3.353272,-0.698750,0.566398,1.816454


In [7]:
model_10

,x1(10),x2(10),x3(10),v1(10),v2(10),v3(10),y1(10),y2(10),y3(10)
0,0.923533,0.995677,-0.879002,-0.879735,0.427313,19.379189,-1.008850,0.643760,16.334485
1,-0.675274,1.000000,-0.921154,1.619019,-0.587812,0.171899,-0.342743,0.744675,-0.105526
2,-0.874069,0.996213,0.820174,186.536532,0.186820,-0.821109,-0.004381,0.430970,0.654714
3,-0.477625,0.999973,0.590695,1.403617,0.401486,5.602000,-0.235170,0.625129,-2.432471
4,0.248300,0.999921,-0.710022,0.841522,0.573865,17.638533,0.147028,0.736805,3.981784
...,...,...,...,...,...,...,...,...,...
95,-0.264963,0.999641,-0.483750,-1.876610,0.592389,7.819336,0.072678,0.747224,-1.884102
96,0.683732,0.999956,0.964232,3.124692,0.295459,-0.927362,0.180935,0.539610,-0.576448
97,-0.517111,0.993039,0.810504,-0.224563,-0.164781,9.807663,1.655909,0.405013,-4.639954
98,-0.691239,0.999393,-0.989284,0.153212,-0.558796,-1.088817,-3.751012,0.728125,0.684589


##1. Линейная форма функции пересчёта

In [8]:
#Построение линейной регрессионной модели
def buildLinearRegretion(model1, model2, cols):
  vector_ones = [1] * elem
  estimate_linear = []
  variance_linear = []
  error_linear = []

  linearFunction = (model1.copy())[cols[0:(len(cols) - 1)]]
  linearFunction[cols[len(cols) - 1]] = (model2.copy())[cols[len(cols) - 1]]
  linearFunction.insert(0, "optional_col", vector_ones)

  A_linear = np.array(linearFunction)
  #A_psev = BenIsrael(A_linear[:, 0:(len(A_linear[0]) - 1)])
  X_linear = solvingSLAE(A_linear[:, 0:(len(A_linear[0]) - 1)], np.matrix(A_linear[:, len(A_linear[0]) - 1]).T)

  for i in range(len(linearFunction)):
    for j in range(0, len(X_linear)):
      if(j == 0):
        estimate_linear.append(X_linear[j,0])
      else:
        estimate_linear[i] += X_linear[j,0] * np.array(linearFunction)[i,j]

    variance_linear.append((1.0 / len(linearFunction)) * (np.array(linearFunction)[i, len(np.array(linearFunction)[0]) - 1] - estimate_linear[i])**2)
    #error_linear.append((1.0 / len(linearFunction)) * abs((np.array(linearFunction)[i, len(np.array(linearFunction)[0]) - 1] - estimate_linear[i])
                            #/ np.array(linearFunction)[i, len(np.array(linearFunction)[0]) - 1]) * 100)
    error_linear.append((1.0 / len(linearFunction)) * abs((np.array(linearFunction)[i, len(np.array(linearFunction)[0]) - 1] - estimate_linear[i])
                            / max(np.array(linearFunction)[:, len(np.array(linearFunction)[0]) - 1])) * 100)

  linearFunction['regression'] = estimate_linear
  linearFunction['variance'] = variance_linear
  linearFunction['error, %'] = error_linear

  return X_linear, linearFunction

###Для функции $x_1[t+1]$

In [9]:
#x1, v1+
X1_linear, X1_linearTable = buildLinearRegretion(model_9, model_10, ['x1(9)', 'v1(9)', 'x1(10)'])
print(pd.DataFrame(X1_linear, columns=['X1'], index=['a1', 'c11', 'b11']))
X1_linearTable

           X1
a1   0.023370
c11  0.198895
b11  0.000064


,optional_col,x1(9),v1(9),x1(10),regression,variance,"error, %"
0,1,0.951079,6.477269,0.923533,0.212951,0.005049,0.714273
1,1,0.885121,31.089224,-0.675274,0.201411,0.007686,0.881238
2,1,-0.056873,1.608868,-0.874069,0.012162,0.007854,0.890833
3,1,-0.741657,1.446732,-0.477625,-0.124049,0.001250,0.355412
4,1,0.194237,3.557893,0.248300,0.062231,0.000346,0.187035
...,...,...,...,...,...,...,...
95,1,0.483175,5.194038,-0.264963,0.119805,0.001480,0.386767
96,1,-0.936859,177.883398,0.683732,-0.151557,0.006977,0.839628
97,1,0.139984,-1.675065,-0.517111,0.051105,0.003229,0.571167
98,1,0.584783,-0.639987,-0.691239,0.139640,0.006904,0.835195


In [10]:
print("E = " + str(sum(X1_linearTable['variance'])))
print("A = " + str(sum(X1_linearTable['error, %'])))

E = 0.46920992410930445
A = 61.85389070580368


###Для функции $x_2[t+1]$

In [11]:
#v2+
X2_linear, X2_linearTable = buildLinearRegretion(model_9, model_10, ['v2(9)', 'x2(10)'])
print(pd.DataFrame(X2_linear, columns=['X2'], index=['a2', 'b22']))
X2_linearTable

           X2
a2   0.997880
b22  0.003647


,optional_col,v2(9),x2(10),regression,variance,"error, %"
0,1,-0.453093,0.995677,0.996227,3.027355e-09,0.000550
1,1,-0.050094,1.000000,0.997697,5.303651e-08,0.002303
2,1,-0.443205,0.996213,0.996263,2.558146e-11,0.000051
3,1,-0.194096,0.999973,0.997172,7.847962e-08,0.002801
4,1,0.232657,0.999921,0.998728,1.421993e-08,0.001192
...,...,...,...,...,...,...
95,1,0.299201,0.999641,0.998971,4.494146e-09,0.000670
96,1,0.210993,0.999956,0.998649,1.707378e-08,0.001307
97,1,-0.490576,0.993039,0.996091,9.314953e-08,0.003052
98,1,-0.326581,0.999393,0.996689,7.315809e-08,0.002705


In [12]:
print("E = " + str(sum(X2_linearTable['variance'])))
print("A = " + str(sum(X2_linearTable['error, %'])))

E = 1.674608310609034e-05
A = 0.28561045634901316


###Для функции $x_3[t+1]$

In [13]:
#x1, x2, v3+
X3_linear, X3_linearTable = buildLinearRegretion(model_9, model_10, ['x1(9)', 'x2(9)', 'v3(9)', 'x3(10)'])
print(pd.DataFrame(X3_linear, columns=['X3'], index=['a3', 'c31', 'c32', 'b33']))
X3_linearTable

           X3
a3  -4.994548
c31 -0.360602
c32  4.767059
b33  0.025760


,optional_col,x1(9),x2(9),v3(9),x3(10),regression,variance,"error, %"
0,1,0.951079,0.995348,0.829883,-0.879002,-0.571248,0.000947,0.307942
1,1,0.885121,0.991708,8.085761,-0.921154,-0.377909,0.002951,0.543577
2,1,-0.056873,0.999978,4.950907,0.820174,-0.079551,0.008095,0.900275
3,1,-0.741657,0.995284,1.414183,0.590695,0.053900,0.002881,0.537123
4,1,0.194237,0.998147,-0.012260,-0.710022,-0.306679,0.001627,0.403589
...,...,...,...,...,...,...,...,...
95,1,0.483175,1.000000,1.047673,-0.483750,-0.374736,0.000119,0.109081
96,1,-0.936859,0.999975,1.171087,0.964232,0.140393,0.006787,0.824343
97,1,0.139984,0.999856,7.887361,0.810504,-0.075478,0.007850,0.886524
98,1,0.584783,0.979541,3.353272,-0.989284,-0.449512,0.002914,0.540103


In [14]:
print("E = " + str(sum(X3_linearTable['variance'])))
print("A = " + str(sum(X3_linearTable['error, %'])))

E = 0.45788771367111575
A = 59.583033942539686


###Для функции $y_1[t]$

In [15]:
#x1, v1+
Y1_linear, Y1_linearTable = buildLinearRegretion(model_9, model_9, ['x1(9)', 'v1(9)', 'y1(9)'])
print(pd.DataFrame(Y1_linear, columns=['Y1'], index=['a1', 'c11', 'b11']))
Y1_linearTable

           Y1
a1   1.906678
c11 -5.067386
b11 -0.000458


,optional_col,x1(9),v1(9),y1(9),regression,variance,"error, %"
0,1,0.951079,6.477269,0.143197,-2.915773,0.093573,0.015269
1,1,0.885121,31.089224,0.026785,-2.592818,0.068623,0.013075
2,1,-0.056873,1.608868,-0.008430,2.194139,0.048513,0.010994
3,1,-0.741657,1.446732,-0.441486,5.664278,0.372803,0.030476
4,1,0.194237,3.557893,0.024061,0.920772,0.008041,0.004476
...,...,...,...,...,...,...,...
95,1,0.483175,5.194038,0.064662,-0.544138,0.003706,0.003039
96,1,-0.936859,177.883398,-0.005098,6.572580,0.432658,0.032832
97,1,0.139984,-1.675065,-0.031267,1.198091,0.015113,0.006136
98,1,0.584783,-0.639987,-0.698750,-1.056352,0.001279,0.001785


In [16]:
print("E = " + str(sum(Y1_linearTable['variance'])))
print("A = " + str(sum(Y1_linearTable['error, %'])))

E = 409.10848386447026
A = 2.732929954413844


###Для функции $y_2[t]$

In [17]:
#v2+
Y2_linear, Y2_linearTable = buildLinearRegretion(model_9, model_9, ['v2(9)', 'y2(9)'])
print(pd.DataFrame(Y2_linear, columns=['Y2'], index=['a2', 'b22']))
Y2_linearTable

           Y2
a2   0.496286
b22 -0.095229


,optional_col,v2(9),y2(9),regression,variance,"error, %"
0,1,-0.453093,0.661625,0.539433,1.493101e-04,0.163828
1,1,-0.050094,0.223769,0.501056,7.688782e-04,0.371769
2,1,-0.443205,0.654856,0.538491,1.354083e-04,0.156015
3,1,-0.194096,0.439180,0.514769,5.713662e-05,0.101345
4,1,0.232657,0.480170,0.474130,3.648228e-07,0.008098
...,...,...,...,...,...,...
95,1,0.299201,0.542915,0.467793,5.643357e-05,0.100719
96,1,0.210993,0.457636,0.476193,3.443707e-06,0.024880
97,1,-0.490576,0.686392,0.543002,2.056050e-04,0.192248
98,1,-0.326581,0.566398,0.527386,1.521995e-05,0.052306


In [18]:
print("E = " + str(sum(Y2_linearTable['variance'])))
print("A = " + str(sum(Y2_linearTable['error, %'])))

E = 0.02411814873427316
A = 16.438652818115383


###Для функции $y_3[t]$

In [19]:
#x1, x2, v3+
Y3_linear, Y3_linearTable = buildLinearRegretion(model_9, model_9, ['x1(9)', 'x2(9)', 'v3(9)', 'y3(9)'])
print(pd.DataFrame(Y3_linear, columns=['Y3'], index=['a3', 'c31', 'c32', 'b33']))
Y3_linearTable

           Y3
a3   6.309891
c31  3.379958
c32 -6.538752
b33  0.159617


,optional_col,x1(9),x2(9),v3(9),y3(9),regression,variance,"error, %"
0,1,0.951079,0.995348,0.829883,0.720578,3.148625,0.058954,0.213665
1,1,0.885121,0.991708,8.085761,6.555670,4.107658,0.059928,0.215422
2,1,-0.056873,0.999978,4.950907,-0.255981,0.369304,0.003910,0.055024
3,1,-0.741657,0.995284,1.414183,-0.957596,-2.479064,0.023149,0.133888
4,1,0.194237,0.998147,-0.012260,-0.002169,0.437810,0.001936,0.038718
...,...,...,...,...,...,...,...,...
95,1,0.483175,1.000000,1.047673,0.460191,1.571479,0.012350,0.097792
96,1,-0.936859,0.999975,1.171087,-0.997425,-3.208315,0.048880,0.194556
97,1,0.139984,0.999856,7.887361,1.003866,1.504181,0.002503,0.044027
98,1,0.584783,0.979541,3.353272,1.816454,2.416696,0.003603,0.052821


In [20]:
print("E = " + str(sum(Y3_linearTable['variance'])))
print("A = " + str(sum(Y3_linearTable['error, %'])))

E = 7.071053261532436
A = 16.629753977659995


##2. Квадратичная форма функции пересчёта

In [39]:
#Построение квадратичной регрессионной модели
def buildQuadraticRegretion(model1, model2, cols):
  vector_ones = [1] * elem
  estimate = []
  variance = []
  error = []

  #переменные
  Function = (model1.copy())[cols[0:(len(cols) - 1)]]
  i = 0
  name_col = ''
  while(i < len(cols) - 1):
    temporary_cols = []
    j = i

    while(j < len(cols) - 1):
      name_col = cols[i] + ' * ' + cols[j]
      for k in range(len(model1)):
        temporary_cols.append(Function[cols[i]][k] * Function[cols[j]][k])

      Function[name_col] = temporary_cols
      temporary_cols.clear()
      j += 1

    i += 1

  #значение функции
  Function[cols[len(cols) - 1]] = (model2.copy())[cols[len(cols) - 1]]

  #доп. столбец
  Function.insert(0, "optional_col", vector_ones)

  A = np.array(Function)
  #A_psev = BenIsrael(A[:, 0:(len(A[0]) - 1)])
  X = solvingSLAE(A[:, 0:(len(A[0]) - 1)], np.matrix(A[:, len(A[0]) - 1]).T)

  for i in range(len(Function)):
    for j in range(0, len(X)):
      if(j == 0):
        estimate.append(X[j,0])
      else:
        estimate[i] += X[j,0] * np.array(Function)[i,j]

    variance.append((1.0 / len(Function)) * (np.array(Function)[i, len(np.array(Function)[0]) - 1] - estimate[i])**2)
    #error_linear.append((1.0 / len(Function)) * abs((np.array(Function)[i, len(np.array(Function)[0]) - 1] - estimate[i])
                            #/ np.array(Function)[i, len(np.array(Function)[0]) - 1]) * 100)
    error.append((1.0 / len(Function)) * abs((np.array(Function)[i, len(np.array(Function)[0]) - 1] - estimate[i])
                            / max(np.array(Function)[:, len(np.array(Function)[0]) - 1])) * 100)

  Function['regression'] = estimate
  Function['variance'] = variance
  Function['error, %'] = error

  return X, Function

###Для функции $x_1[t + 1]$

In [40]:
#x1, v1+
X1_quadratic, X1_quadraticTable = buildQuadraticRegretion(model_9, model_10, ['x1(9)', 'v1(9)', 'x1(10)'])
print(pd.DataFrame(X1_quadratic, columns=['X1'], index=['a1', 'c11', 'b11', 'd111', 'd113', 'd112']))
X1_quadraticTable

                X1
a1    3.428710e-02
c11   1.773146e-01
b11   4.501198e-04
d111 -5.014247e-02
d113  2.195750e-04
d112 -7.356998e-08


,optional_col,x1(9),v1(9),x1(9) * x1(9),x1(9) * v1(9),v1(9) * v1(9),x1(10),regression,variance,"error, %"
0,1,0.951079,6.477269,0.904551,6.160393,41.955017,0.923533,0.161836,0.005802,0.765653
1,1,0.885121,31.089224,0.783439,27.517721,966.539843,-0.675274,0.171913,0.007177,0.851588
2,1,-0.056873,1.608868,0.003235,-0.091501,2.588458,-0.874069,0.024744,0.008079,0.903481
3,1,-0.741657,1.446732,0.550055,-1.072979,2.093035,-0.477625,-0.124385,0.001248,0.355074
4,1,0.194237,3.557893,0.037728,0.691076,12.658606,0.248300,0.068589,0.000323,0.180645
...,...,...,...,...,...,...,...,...,...,...
95,1,0.483175,5.194038,0.233458,2.509631,26.978031,-0.264963,0.111142,0.001415,0.378059
96,1,-0.936859,177.883398,0.877704,-166.651577,31642.503310,0.683732,-0.134693,0.006698,0.822676
97,1,0.139984,-1.675065,0.019596,-0.234483,2.805841,-0.517111,0.057320,0.003300,0.577415
98,1,0.584783,-0.639987,0.341972,-0.374254,0.409583,-0.691239,0.120460,0.006589,0.815915
